# Mavlink  Base_1
> Mavlink  base class for Client and server.
> Theserver is on the companion computer and the client is on the ground station PC.

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE



In [ ]:
#| default_exp mavlink.base_1

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time, os, sys

from UAV.logging import logging
from UAV.utils.general import LeakyQueue

# os.environ['MAVLINK20'] == '1' should be placed in UAV.__init__.py
assert os.environ['MAVLINK20'] == '1', "Set the environment variable before from pymavlink import mavutil  library is imported"

# logging.getLogger("uav").setLevel(logging.DEBUG)
# logging.root.setLevel(logging.INFO)
import threading
import queue
import typing as typ
from pathlib import Path
from inspect import currentframe, getframeinfo
from pymavlink import mavutil
# from UAV.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
# logging.getLogger("uav").setLevel(logging.INFO)

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

def get_linenumber():
    cf = currentframe()
    filename = Path(getframeinfo(cf).filename).name
    return  f"{filename}:{cf.f_back.f_lineno}"

def format_rcvd_msg(msg, extra='')  :
    """ Format a message for logging."""
    s = f"{str(msg)} ... {extra}"
    try:
        s = f"Rcvd message from: {msg.get_srcSystem():3d}, comp: {msg.get_srcComponent():3d} {s}"
    except:
        try:
            s = f"Rcvd message from: {'???'}, comp: {msg.get_srcComponent():3d} {s}"
        except:
            s = f"Rcvd message from: {'???'}, comp: {'???'} {s}"
    return s


class Component:
    """Create a mavlink Component with an ID  for MAV_COMPONENT"""
    def __init__(self, mav_connection, # MavLinkBase connection
                 source_component, # used for component indication
                 # target_system, # target_component
                 # target_component, # target_component
                 mav_type, # used for heartbeat MAV_TYPE indication
                 debug): # logging level
        # todo change to def __init__(self:MavLinkBase, ....
        self.mav_connection:MavLinkBase = mav_connection
        self.master = mav_connection.master
        self.mav_type = mav_type
        self.source_system = self.mav_connection.source_system
        self.source_component = source_component
        self.target_system = None  # Request ping of this system
        self.target_component = None  # Request ping of this component
        
        self._log = logging.getLogger("uav.{}".format(self.__class__.__name__))
        self._log.setLevel(logging.DEBUG if debug else logging.INFO)
        
        self.ping_num = 0
        self.num_commands_sent = 0
        self.num_commands_rcvd = 0
        self.num_msgs_received = 0
        self.num_acks_received = 0
        self.message_cnts:{} = {} # received message counts, indexed by system and message type

        self._heartbeat_que = LeakyQueue(maxsize=10)
        self._ack_que = LeakyQueue(maxsize=10)
        self._message_que = LeakyQueue(maxsize=10)


        self._t_heartbeat = threading.Thread(target=self.send_heartbeat, daemon=True)
        self._t_heartbeat.start()

        self._t_command = threading.Thread(target=self.listen, daemon=True)
        self._t_command.start()
        self.log.info(f"Component Started {self.source_component = }, {self.mav_type = }, {self.source_system = }, {self.target_system = }, {self.target_component = } ")

        
    def __str__(self) -> str:
        return self.__class__.__name__
    

    def __repr__(self) -> str:
        return "<{}>".format(self)
    
    
    @property
    def log(self) -> logging.Logger:
        return self._log

    def set_source_compenent(self):
        """Set the source component for the master.mav """
        self.master.mav.srcComponent = self.source_component

    def send_ping(self, target_system:int, target_component:int):
        """Send a ping message to test if the server is alive."""
        try:
            self.ping_num += 1
        except:
            self.ping_num = 0

        self.set_source_compenent()
        self.master.mav.ping_send(
            int(time.time() * 1000),  # Unix time 
            self.ping_num,  # Ping number
            target_system,  # Request ping of this system
            target_component,  # Request ping of this component
        )   
        self.log.debug(f"Sent Ping #{self.ping_num} to:   {target_system:3d}, comp: {target_component:3d}")



    def send_heartbeat(self):
        """Send a heartbeat message to indicate the server is alive."""
        self._t_heartbeat_stop = False

        # self.log.info(f"Starting heartbeat type: {self.mav_type} to all Systems and Components")
        while not self._t_heartbeat_stop:
            self.set_source_compenent()
            self.log.debug(f"Sent hrtbeat to All")
                         # "Sent Ping #2 to:   111, comp: 100"
            self.master.mav.heartbeat_send(
                self.mav_type,  # type
                # mavutil.mavlink.MAV_TYPE_ONBOARD_CONTROLLER,
                mavutil.mavlink.MAV_AUTOPILOT_INVALID,  # autopilot
                0,  # base_mode
                0,  # custom_mode
                mavutil.mavlink.MAV_STATE_ACTIVE,  # system_status
            )
            # print("Cam heartbeat_send")
            time.sleep(1)  # Send every second
    

    def wait_heartbeat(self, remote_mav_type=None, # type of remote system
                       timeout:int=1, # seconds
                       tries:int=3)->bool: # number of tries
        """Wait for an heartbeat from target_system and target_component."""
        # Todo is this correct ? Wait for a heartbeat, so we know the target system IDs (also it seems to need it to start receiving commands)
        if remote_mav_type is None:
            self.log.debug(f"Waiting for heartbeat from system: {self.target_system} comp: {self.target_component}")
        else:
            self.log.debug(f"Waiting for heartbeat type: {remote_mav_type} from system: {self.target_system} comp: {self.target_component}")
        count = 0
        while count < tries:
            try:
                msg = self._heartbeat_que.get(timeout=timeout)
                self.log.debug(format_rcvd_msg(msg, extra='self._heartbeat_que.get() '))
                # self.log.debug(f"Rcvd Heartbeat from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg} ")
                # check if the heartbeat is from the correct system and component
                if msg.type == remote_mav_type and msg.get_srcSystem() == self.target_system and msg.get_srcComponent() == self.target_component:
                    return True
                elif msg.get_srcSystem() == self.target_system and msg.get_srcComponent() == self.target_component:
                    return True
            except queue.Empty:  # i.e time out
                count += 1

        self.log.debug(f"No heartbeat received after {tries} tries")
        return False
    
            
    def send_ack(self, msg, ack_result=mavutil.mavlink.MAV_RESULT_ACCEPTED):
        """Send an ACK message to indicate a command was received."""
        self.set_source_compenent()
        self.master.mav.command_ack_send(
                    msg.command,
                    ack_result,  # or other MAV_RESULT enum
                    # todo enabling these causes QGC not to show them
                    int(0),  # progress
                    int(0),  # result_param2
                    self.target_system,  # target_system
                    self.target_component,  # target_component
        )

        
    def wait_ack(self, mav_cmd=None, timeout:int=1, # seconds
                       tries:int=3)->bool: # number of tries
        """Wait for an ack from target_system and target_component."""
        self.log.debug(f"Waiting for ACK for command: {mav_cmd} from system: {self.target_system} comp: {self.target_component}")
        count = 0
        while count < tries:
            try:
                msg = self._ack_que.get(timeout=timeout)
                self.log.debug(f"ACK received from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                if mav_cmd ==  msg.command and msg.get_srcSystem() == self.target_system and msg.get_srcComponent() == self.target_component:
                    self.log.debug(f"ACK received from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                    return True
                elif msg.get_srcSystem() == self.target_system and msg.get_srcComponent() == self.target_component:
                    self.log.debug(f"ACK received from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                    return True
                else:
                    self.log.debug(f"*** ACK not handled {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
            except queue.Empty:  # i.e time out
                count += 1
        self.log.debug("No ACK received")
        return False

    def count_message(self, msg):
        """ Count a message by adding it to the message_cnts dictionary. indexed by system and message type"""
        try:
            self.message_cnts[msg.get_srcSystem()][msg.get_type()] += 1
        except Exception as e:
            # print(f"!!!! new Message type {msg.get_type()} from system {msg.get_srcSystem()}")
            sys = msg.get_srcSystem()
            if sys not in self.message_cnts:
                self.message_cnts[sys] = {}
            self.message_cnts[sys][msg.get_type()] = 1

        return True
    # def log_message_rcvd(self, msg, extra='', debug=False, hide=False):
    #     """ Log a message."""
    #     if not hide:
    #         s = f"Rcvd message from: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {extra} {msg}"
    #         self.log.debug(s) if debug else self.log.info(s)


    def listen(self, timeout:int=1, ):  # seconds
        """Listen for MAVLink commands and trigger the camera when needed."""
        # assert hasattr(self, 'master'), "start_mavlink() must be called before threading.Thread(target=self.listen..."

        self._t_cmd_listen_stop = False
        # self.log.info(f"Component Listening for messages sent on the message_queue ...")
        while not self._t_cmd_listen_stop:

            try:
                msg = self._message_que.get(timeout=timeout)
                self.log.debug(format_rcvd_msg(msg))
                # log_message_rcvd(self, msg)
                # self.log.debug(f"Rcvd message from: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                # if command_type ==  msg.command and msg.get_srcSystem() == self.target_system and msg.get_srcComponent() == self.target_component:
                #     self.log.info(f"ACK received from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                #     return True
                # elif msg.get_srcSystem() == self.target_system and msg.get_srcComponent() == self.target_component:
                #     self.log.info(f"ACK received from src_sys: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {msg}")
                #     return True
                self.num_msgs_received += 1
            except queue.Empty:  # i.e time out

                time.sleep(0.01)
                continue

            self.count_message(msg)

            if msg.get_type() == 'COMMAND_LONG':
                self.on_command_rcvd(msg)
                # self.num_msgs_received += 1
            elif msg.get_type() == 'COMMAND_INT':
                self.on_command_rcvd(msg)
                # self.num_msgs_received += 1

            elif msg.get_type() == 'COMMAND_ACK':
                # self.log.debug(f"Received ACK ")
                # print(f"***  {get_linenumber()}:   ack_que.put  {msg}")
                self._ack_que.put(msg, block=False)
                self.num_acks_received += 1

            elif msg.get_type() == 'HEARTBEAT':
                # self.log.debug(f"Received HEARTBEAT ")
                self._heartbeat_que.put(msg, block=False)


            elif msg.get_type() == 'PING':
                self.log.debug(f"Received PING {msg}")
                if self.mav_connection.is_server:  # only servers respond to pings
                    print(f"Todo self.send_ping() {get_linenumber()}")
                    # self.component_list[0].send_ping()


    def on_command_rcvd(self, msg):
        # Callback for when a command is received.
        print(f"!!! YAY!!! {get_linenumber()} {self} Received command {msg}, sending ACK")
        self.num_commands_rcvd += 1
        self.send_ack(msg,  mavutil.mavlink.MAV_RESULT_ACCEPTED)


    def send_command(self, target_system:int, # target system
                         target_component:int, # target component
                         command_id: int, # mavutil.mavlink.MAV_CMD....
                         params: list,): # list of parameters
        self.log.debug(f"Sending command: {command_id} to system: {self.target_system} comp: {self.target_component}")
        self.set_source_compenent()
        command_id = self.master.mav.command_long_send(
            target_system,  # target_system   Todo Tried using self.master.target_system but it didn't work
            target_component,  # target_component Todo tried using self.master.target_component but it didn't work
            command_id,  # command id
            0,  # confirmation
            *params  # command parameters
        )
        # self.count_message(msg)
        self.num_commands_sent += 1

        return self.wait_ack(command_id, timeout=0.1)


    def _test_command(self, target_system:int, # target system
                            target_component:int, # target component
                            camera_id:int=1): # camera id (0 for all cams)
        """
        Use MAV_CMD_DO_DIGICAM_CONTROL to trigger a camera 
        """
        self.set_source_compenent()
        mav_cmd = mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL
        rst = self.send_command(target_system, target_component,
            mav_cmd,
                          [camera_id,  # param1 (session)  or cam # (0 for all cams)
                           1,  # param2 (trigger capture)
                           0,  # param3 (zoom pos)
                           0,  # param4 (zoom step)
                           0,  # param5 (focus lock)
                           0,  # param6 (shot ID)
                           0,  # param7 (command ID)
                          ])
        # self.listen_for_ack()
        # self.log.debug(f"Sent message to:   {self.target_system:3d}, comp: {self.target_component:3d} command MAV_CMD_DO_DIGICAM_CONFIGURE")
        # self.num_commands_sent -= 1   # don't count test command
        return rst

    def close(self):
        # print(f"Closing {self.__class__.__name__}...")

        self._t_heartbeat_stop = True
        self._t_heartbeat.join()
        

In [ ]:
#| export
MAV_SYSTEM_GCS_CLIENT = 200  # GCS type client (TODO its not clear if this is correct,  255 = GCS)
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_SYSTEM_VEHICLE = 111   # 1 = vehicle
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
MAV_COMP_ID_CAMERA = mavutil.mavlink.MAV_COMP_ID_CAMERA
MAV_COMP_ID_USER1 = mavutil.mavlink.MAV_COMP_ID_USER1


In [ ]:
#| export
class Client_Comp(Component):
    """Create a mavlink Client Component of type  MAV_TYPE_GCS"""
    def __init__(self,
                    mav_connection=None, # MavLinkBase connection
                    source_component=11, # used for component indication
                    mav_type=MAV_TYPE_GCS, # used for heartbeat MAV_TYPE indication
                    debug=False): # logging level
            super().__init__(mav_connection=mav_connection, source_component=source_component,      mav_type=mav_type, debug=debug)
            self.target_system = 222  # Request ping of this system
            self.target_component = 22  # Request ping of this component
        
class Srv_Comp1(Component):
    """Create a mavlink Camera Component of type  MAV_TYPE_CAMERA"""
    def __init__(self,
                    mav_connection=None, # MavLinkBase connection
                    source_component=22, # used for component indication
                    mav_type=MAV_TYPE_CAMERA, # used for heartbeat MAV_TYPE indication
                    debug=False): # logging level
            super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type, debug=debug)
            self.target_system = 111  # system to relpy to (i.e. the client)
            self.target_component = 11  # component to relpy to (i.e. the client)


class Srv_Comp2(Component):
    """Create a mavlink Camera Component of type  MAV_TYPE_CAMERA"""
    def __init__(self,
                 mav_connection=None,  # MavLinkBase connection
                 source_component=22+1,  # used for component indication
                 mav_type=MAV_TYPE_CAMERA,  # used for heartbeat MAV_TYPE indication
                 debug=False):  # logging level
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type, debug=debug)
        self.target_system = 111  # system to relpy to (i.e. the client)
        self.target_component = 11  # component to relpy to (i.e. the client)


In [ ]:
#| export

class MavLinkBase:
    """
    Mavlink Base 
    """
    def __init__(self, connection_string,  # "udpin:localhost:14550"
                 baudrate=57600,  #baud rate of the serial port
                 source_system = MAV_SYSTEM_VEHICLE,  # remote or air uav system   1 = vehicle
                 target_system = MAV_SYSTEM_GCS_CLIENT,  # GCS system   255 = GCS
                 # mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, # type used in heartbeat
                 debug=False,  # logging level
                 ):

        self._log = logging.getLogger("uav.{}".format(self.__class__.__name__))
        if debug:
            log_level = logging.DEBUG
        else:
            log_level = logging.INFO
        self._log.setLevel(log_level)
        self.connection_string:str = connection_string
        self.baudrate:int = baudrate
        self.source_system:int = source_system
        self.target_system:int = target_system
        self.debug = debug

        # self.num_commands_received:int = 0
        # self.num_acks_received:int = 0
        
        self.check_message_route(None)
        # self.ping_num = 0
        # self.num_commands_received = 0
        # self.num_commands_sent = 0
        # self.num_acks_received = 0
        self.message_cnts:{} = {} # received message counts, indexed by system and message type

        # self._heartbeat_que = LeakyQueue(maxsize=10)
        # self._ack_que = LeakyQueue(maxsize=10)
        self.component:{Component} = {}
        self._t_mav_listen_stop = True
        self._t_mav_listen = None



    def server(self, 
               # mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, # type used in heartbeat
               #      source_system = None, # source system   1 = vehicle, 195 = companion computer
                    target_system = None,  # target system   1 = vehicle, 195 = companion computer
                    # source_component = mavutil.mavlink.MAV_COMP_ID_CAMERA, # source component
                    target_component = mavutil.mavlink.MAV_COMP_ID_USER1, # target component
                    # do_heartbeat:bool=True, # send heartbeat
                    # do_listen:bool=True, # listen for commands
                    do_ack:bool=True): # send ack
        """Set the server defaults."""
        assert not hasattr(self, 'is_server'), "server() can only be called once"
        self.is_server = True
        # self.mav_type = mav_type

        # self.source_system = self.source_system if source_system is None else source_system
        # self.target_system = self.client_system_ID if target_system is None else target_system
        # self.source_component = source_component
        # self.target_component = target_component


        # self.do_heartbeat = do_heartbeat
        # self.do_listen = do_listen
        self.do_ack = do_ack
        self.start_mavlink()
        return self
    
    def client(self, 
               # mav_type=mavutil.mavlink.MAV_TYPE_GCS, # type used in heartbeat
               #      source_system=None,  # source system   1 = vehicle, 195 = companion computer
                    target_system=None,  # target system   1 = vehicle, 195 = companion computer
                    # source_component = mavutil.mavlink.MAV_COMP_ID_USER1, # source component
                    target_component = mavutil.mavlink.MAV_COMP_ID_CAMERA, # target component
                    # do_heartbeat:bool=True, # send heartbeat
                    # do_listen:bool=True, # listen for commands
                    do_ack:bool=False):  # send ack
        """Set client defaults."""
        assert not hasattr(self, 'is_server'), "client() can only be called once"
        self.is_server = False

        # self.mav_type = mav_type

        # self.source_system = self.target_system     if source_system is None else source_system
        # self.target_system = self.server_system_ID  if target_system is None else target_system
        # self.source_component = source_component
        # self.target_component = target_component


        # self.do_heartbeat = do_heartbeat
        # self.do_listen = do_listen
        self.do_ack = do_ack
        self.start_mavlink()
        return self
        
    
    def start_mavlink(self):
        """Start the MAVLink connection."""
        # Create the connection  Todo add source_system and component options
        # self.log.info(f"Starting MAVLink connection... Mavlink version 2 = {mavutil.mavlink20()}")
        # self._ack_evt.clear()
        # self._heartbeat_evt.clear()

        self.master = mavutil.mavlink_connection(self.connection_string, # "udpin:localhost:14550"
                                                 baud=self.baudrate, # baud rate of the serial port
                                                 source_system=int(self.source_system), # source system
                                                 source_component=169, # source component
                                                 )
                                                 # )

        self.master.target_system = self.target_system  # Todo master.target_system gets changed by heart beat
        # Todo master.target_system gets changed by heart beat
        print(f"{get_linenumber()} {self.master.source_component = }")
        

        # self.log.info(f"Target system Set: {self.master.target_system}, Target component: {self.master.target_component}")
        # self.log.info(f"see https://mavlink.io/en/messages/common.html#MAV_COMPONENT")
        time.sleep(0.1)  # Todo delay for connection to establish
        # if self.do_heartbeat:
        #     self._t_heartbeat = threading.Thread(target=self.send_heartbeat, daemon=True)
        #     self._t_heartbeat.start()

        assert hasattr(self, 'master'), "start_mavlink() must be called before threading.Thread(target=self.listen..."

        self._t_mav_listen = threading.Thread(target=self.listen, daemon=True)

        # print(f"Target system: {self.target_system}, Target component: {self.target_component}")

        assert self.master.mavlink20(), "Mavlink 2 protocol is not happening ?, check os.environ['MAVLINK20'] = '1'"

        # self.component_list.append(Comp_CamManager(self, self.server_system_ID, debug=self.debug))  # add q default source component

    def start_listen(self):
        """Listen for MAVLink commands """
        self._t_mav_listen.start()

    def __str__(self) -> str:
        return self.__class__.__name__

    def __repr__(self) -> str:
        return "<{}>".format(self)

    @property
    def log(self) -> logging.Logger:
        return self._log
    
    # def log_message_rcvd(self, msg, extra='', debug=False, hide=False):
    #     """ Log a message."""
    #     if not hide:
    #         s = f"{str(msg):20s}"
    #         s = f"Rcvd message from: {msg.get_srcSystem()}, src_comp: {msg.get_srcComponent()} {s}"
    #         self.log.debug(s) if debug else self.log.info(s)
    def check_message_route(self, msg,  # message
                            hide_log=False,  # hife log output for messages
                            )->typ.Union[int,None]:
        """
               check message and routing. return the component ID to be routed to , 0 = all, None = none.

                Systems/components should process a message if any of these conditions hold:
                   - It is a broadcast message (target_system field omitted or zero)
                   - The target_system matches its system id and target_component is broadcast (target_component omitted or zero).
                   - The target_system matches its system id and has the component's target_component or
                   - The target_system matches its system id and the component is unknown (i.e. this component has not seen any messages on any link that have the message's target_system/target_component)
               """


        if msg is None:
            return None

        self.count_message(msg)

        if msg.get_type() == "BAD_DATA":
            return None

        # Test - It is a broadcast message (target_system field omitted or zero)
        if not hasattr(msg, 'target_system') or msg.target_system == 0:
            self.log.debug(format_rcvd_msg(msg, "target_system: ???, Pass to ALL Components"))
            return 0 #  for all components

        # Test - The target_system matches its system id and target_component is broadcast (target_component omitted or zero).
        #      - The target_system matches its system id and has the component's target_component
        if msg.target_system == self.source_system:
            if hasattr(msg, 'target_component'):
                self.log.debug(format_rcvd_msg(msg, f"Pass to Component {msg.target_component:3d}"))

                return msg.target_component
            else:
                self.log.debug(format_rcvd_msg(msg, " Pass to ALL Components {'  0'}"))
                return 0

        self.log.error(format_rcvd_msg(msg, "!!!!! No match !!!! {msg.target_system} != {self.source_system}"))


    def count_message(self, msg):
        """ Count a message by adding it to the message_cnts dictionary. indexed by system and message type"""
        try:
            self.message_cnts[msg.get_srcSystem()][msg.get_type()] += 1
        except Exception as e:
            # print(f"!!!! new Message type {msg.get_type()} from system {msg.get_srcSystem()}")
            sys = msg.get_srcSystem()
            if sys not in self.message_cnts:
                self.message_cnts[sys] = {}
            self.message_cnts[sys][msg.get_type()] = 1

        return True

    def listen(self):
        """Listen for MAVLink commands and trigger the camera when needed."""
        assert hasattr(self, 'master'), "start_mavlink() must be called before threading.Thread(target=self.listen..."

        self._t_mav_listen_stop = False
        # self.log.info(f"Listening for MAVLink commands from system: {self.target_system}...")
        # self.log.info(f"Listening for MAVLink commands from all systems...")
        self.log.info(f"MAVLink Mav2: {mavutil.mavlink20()}, source_system: {self.source_system}, target_system: {self.target_system}")

        while not self._t_mav_listen_stop:
            # Wait for a MAVLink message
            try:   # Todo: catch bad file descriptor error
                msg = self.master.recv_match(blocking=True, timeout=1)
            except Exception as e:
                self.log.debug(f"Exception: {e}")
                time.sleep(1)
                continue
            comp_ID = self.check_message_route(msg)

            if comp_ID is None:
                continue # don't process message
            if comp_ID == 0:
                # send to all components
                for key, comp in self.component.items():
                    comp._message_que.put(msg, block=False)
            else: # send to specific component
                try:
                    self.component[comp_ID]._message_que.put(msg, block=False)
                except Exception as e:
                    self.log.error(f" Component {comp_ID} does not exist? ; Exception: {e}")
                    continue


    def add_component(self, comp:Component):
        # append a component to the component dictionary with the key being the source_component
        self.component[comp.source_component] = comp
    
        
    def close(self):
        # print(f"Closing {self.__class__.__name__}...")

        self._t_mav_listen_stop = True
        # self._t_heartbeat_stop = True
        # if self.do_heartbeat:
        #     self._t_heartbeat.join()

        self._t_mav_listen.join()
        for key, comp in self.component.items():
            comp.close()

        self.master.close()
        self.master.port.close()
        self.log.info(f"{self.__class__.__name__}  closed")
    
    def __enter__(self):
        """ Context manager entry point for with statement."""
        return self # This value is assigned to the variable after 'as' in the 'with' statement
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Context manager exit point."""
        self.close()
        return False  # re-raise any exceptions

class _Server(MavLinkBase):
    def do_nothing(self):
        pass

class _Client(MavLinkBase):
    def do_nothing(self):
        pass
    
def _test_client_server(debug=False):

# _TEST_HERE = False
# if _TEST_HERE:
    from fastcore.test import test_eq
    # Test sending a command and receiving an ack from client to server
    with _Client("udpin:localhost:14445", source_system_ID=111, target_system_ID=222, debug=debug).client() as client:
        with _Server("udpout:localhost:14445", source_system_ID=111, target_system_ID=222, debug=debug).server() as server:
            client.wait_heartbeat()
            NUM_TO_SEND = 2
            for i in range(NUM_TO_SEND):
                client.send_command(mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL, [0,0,0,0,0,0,0])
                client.wait_ack(timeout=0.1)
                client.send_ping()
                # client.listen_for_ack()
                
            # see if anymore acks come in
            client.wait_ack(timeout=0.1)
    
            print(f"client.num_commands_sent: {client.num_commands_sent}")
            print(f"server.num_commands_received: {server.num_commands_received}")
            print(f"client.num_acks_received: {client.num_acks_received}")

            print(f"server sys: {server.source_system};  msgs: {server.message_cnts}")
            print(f"client sys: {client.source_system};  msgs: {client.message_cnts}")

            test_eq(server.source_system, server.source_system)
            test_eq(client.target_system, client.source_system)
    
            test_eq(client.num_commands_sent, server.num_commands_received)
            test_eq(client.num_acks_received, NUM_TO_SEND)
            test_eq(server.message_cnts[222]['COMMAND_LONG'], client.message_cnts[111]['COMMAND_ACK'])
            test_eq(server.ping_num, client.ping_num)
            assert client.message_cnts[111]['HEARTBEAT'] >= 1

if __name__ == "__main__":
    # test_client_server(debug=True)
    # client_comp = GCS_Component()
    with MavLinkBase("udpin:localhost:14445", debug=True).client() as client:
        client.add_component(Client_Comp(client, debug=True))
        for comp in client.component:
            print(comp)
            result = comp.wait_heartbeat()
            test_eq(result, False)
        # result = client.component_list[0].wait_heartbeat(timeout=0.1)
        # should return no heart beat
        # test_eq(result, False)



INFO   | uav.MavLinkBase      | 16:24:19.188 |[27474574.py:202] Thread-15 (listen) | Listening for MAVLink commands from all systems...
INFO   | uav.MavLinkBase      | 16:24:19.188 |[27474574.py:101] MainThread | Starting MAVLink connection... Mavlink version 2 = True
DEBUG  | uav.MavLinkBase      | 16:24:19.189 |[27474574.py:208] Thread-15 (listen) | Exception: 'MavLinkBase' object has no attribute 'master'
DEBUG  | uav.MavLinkBase      | 16:24:19.189 |[27474574.py:208] Thread-15 (listen) | Exception: 'MavLinkBase' object has no attribute 'master'
INFO   | uav.MavLinkBase      | 16:24:19.190 |[27474574.py:115] MainThread | Source system Set: 200, Source component: 25
INFO   | uav.MavLinkBase      | 16:24:19.190 |[27474574.py:116] MainThread | Target system Set: 0, Target component: 0
INFO   | uav.MavLinkBase      | 16:24:19.291 |[27474574.py:202] Thread-16 (listen) | Listening for MAVLink commands from all systems...
INFO   | uav.Comp_CamManager  | 16:24:19.292 |[1739792039.py: 65] Th

Comp_CamManager


DEBUG  | uav.Comp_CamManager  | 16:24:24.293 |[1739792039.py:101] MainThread | No heartbeat received after 5 tries
INFO   | uav.MavLinkBase      | 16:24:25.315 |[27474574.py:283] MainThread | MavLinkBase  closed


In [ ]:
# | hide
assert False, "Stop here"

AssertionError: Stop here

In [ ]:
show_doc(MavLinkBase)

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
# Test sending a command and receiving an ack from client to server
with _Client("udpin:localhost:14445", server_system_ID=111, client_system_ID=222, debug=False).client() as client:
    with _Server("udpout:localhost:14445", server_system_ID=111, client_system_ID=222, debug=False).server() as server:
        client.wait_heartbeat()
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.send_command(mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL, [0,0,0,0,0,0,0])
            client.wait_ack(timeout=0.1)
            # client.listen_for_ack()
            
        # see if anymore acks come in
        client.wait_ack(timeout=0.1)

    print(f"client.num_commands_sent: {client.num_commands_sent}")
    print(f"server.num_commands_received: {server.num_commands_received}")
    print(f"client.num_acks_received: {client.num_acks_received}")
    
    print(f"server sys: {server.source_system};  msgs: {server.message_cnts}")
    print(f"client sys: {client.source_system};  msgs: {client.message_cnts}")

        
    test_eq(server.server_system_ID, server.source_system)
    test_eq(client.client_system_ID, client.source_system)

    test_eq(client.num_commands_sent, server.num_commands_received)
    test_eq(client.num_acks_received, client.num_commands_sent)
    test_eq(server.message_cnts[222]['COMMAND_LONG'], client.message_cnts[111]['COMMAND_ACK'])
    assert client.message_cnts[111]['HEARTBEAT'] >= 1

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(MavLinkBase.server)

In [ ]:
show_doc(MavLinkBase.client)

In [ ]:
show_doc(MavLinkBase.on_command_rcvd)

In [ ]:
show_doc(MavLinkBase.send_command)

In [ ]:
show_doc(MavLinkBase.wait_heartbeat)

In [ ]:
show_doc(MavLinkBase.wait_ack)

In [ ]:
show_doc(MavLinkBase.send_ping)

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()